# **1. Business Understanding**

## **1.1 Problem Statement**
Individuals and small businesses often struggle to track their **income**, **expenses**, **savings**, and **budgets** in a structured and consistent way. Without a proper financial management system, it becomes difficult to understand where money is going, prevent overspending, or maintain healthy saving habits.

A **Financial Management System (FMS)** helps users organize financial data by storing **transactions**, **budgets**, **accounts**, **categories**, and **savings goals**.  
This project focuses on designing and querying a simple relational database that supports essential financial management features.

## **1.2 Objectives**
The goal of this SQL project is to design a functional **Financial Management System database** and write SQL queries that enable users to:

- Track **income** and **expenses**  
- View **account balances**  
- Categorize transactions (e.g., groceries, rent, salary)  
- Set and monitor **budgets**  
- Track **savings goals**  
- Analyse spending patterns over time  

## **1.3 Key Questions to Answer Using SQL**
The analysis will answer financial questions such as:

- How much did the user spend this month?  
- What are the top categories where money is going?  
- What is the **total income vs total expense** for a given period?  
- Which months had the highest savings?  
- How close is the user to meeting their savings goals?  

## **1.4 Project Scope**
This SQL project will involve:

- Designing the database schema using **ERD principles**  
- Creating database tables in SQL  
- Inserting sample data  
- Running analytical SQL queries  
- Creating **views** for simplified reporting  
- Documenting insights and conclusions  



# **2. Data Understanding**

To build the Financial Management System (FMS), we define a relational database structure that stores essential financial information.  
The system revolves around five main entities: **Users**, **Accounts**, **Categories**, **Transactions**, **Budgets**, and **SavingsGoals**.

Below is a clear description of each table and the role it plays in the system.

## **2.1 Users Table**
Stores information about each user of the financial system.

**Fields:**
- **user_id** (Primary Key)  
- **name**  
- **email**

## **2.2 Accounts Table**
Stores different financial accounts such as bank accounts, mobile money, or cash wallets.

**Fields:**
- **account_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **account_name**  
- **account_type** (e.g., checking, savings, mobile money, cash)  
- **balance**

## **2.3 Categories Table**
Defines categories for organizing transactions.

**Fields:**
- **category_id** (Primary Key)  
- **category_name** (e.g., groceries, rent, utilities, salary)  
- **category_type** (Income or Expense)

## **2.4 Transactions Table**
Stores detailed financial activities (income and expenses).

**Fields:**
- **transaction_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **account_id** (Foreign Key → Accounts.account_id)  
- **category_id** (Foreign Key → Categories.category_id)  
- **amount**  
- **transaction_date**  
- **description**

## **2.5 Budgets Table**
Stores monthly budget limits for each spending category.

**Fields:**
- **budget_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **category_id** (Foreign Key → Categories.category_id)  
- **month**  
- **year**  
- **budget_amount**

## **2.6 SavingsGoals Table**
Tracks user savings goals and progress.

**Fields:**
- **goal_id** (Primary Key)  
- **user_id** (Foreign Key → Users.user_id)  
- **goal_name**  
- **target_amount**  
- **current_amount**  
- **deadline**

## **2.7 Summary**
This data structure ensures the FMS can:

- Store financial activity  
- Support categorization  
- Track budgets  
- Measure savings progress  
- Analyse trends over time  

# **3. Data Preparation**

This step involves creating the full SQL schema for the Financial Management System (FMS) using Python and SQLite.

The following structural components were defined:

- A connection to a SQLite database  
- All required tables for the system  
- Primary keys and foreign keys  
- A database layout capable of supporting income tracking, expense tracking, budgets, and savings goals  

The code block below contains the SQL schema creation.


In [1]:
import sqlite3

# Connect to database (creates a new file if it does not exist)
conn = sqlite3.connect("financial_management_system.db")
cur = conn.cursor()

# Enable foreign keys in SQLite
cur.execute("PRAGMA foreign_keys = ON;")

# 1. Users Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Users (
    user_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL
);
""")

# 2. Accounts Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Accounts (
    account_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    account_name TEXT NOT NULL,
    account_type TEXT NOT NULL,
    balance REAL DEFAULT 0,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
""")

# 3. Categories Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Categories (
    category_id INTEGER PRIMARY KEY,
    category_name TEXT NOT NULL,
    category_type TEXT CHECK (category_type IN ('Income', 'Expense')) NOT NULL
);
""")

# 4. Transactions Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    account_id INTEGER NOT NULL,
    category_id INTEGER NOT NULL,
    amount REAL NOT NULL,
    transaction_date TEXT NOT NULL,
    description TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (account_id) REFERENCES Accounts(account_id),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);
""")

# 5. Budgets Table
cur.execute("""
CREATE TABLE IF NOT EXISTS Budgets (
    budget_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    category_id INTEGER NOT NULL,
    month INTEGER NOT NULL,
    year INTEGER NOT NULL,
    budget_amount REAL NOT NULL,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);
""")

# 6. SavingsGoals Table
cur.execute("""
CREATE TABLE IF NOT EXISTS SavingsGoals (
    goal_id INTEGER PRIMARY KEY,
    user_id INTEGER NOT NULL,
    goal_name TEXT NOT NULL,
    target_amount REAL NOT NULL,
    current_amount REAL DEFAULT 0,
    deadline TEXT,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);
""")

conn.commit()
conn.close()

print("Database schema created successfully.")


Database schema created successfully.
